In [0]:
import re
from tqdm import tqdm
import nltk
from nltk import word_tokenize
nltk.download('tagsets')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
!pip install flair
from flair.data import Sentence
from flair.models import SequenceTagger
from google.colab import files
import io

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
    100% |████████████████████████████████| 71kB 1.8MB/s 
    100% |████████████████████████████████| 591.8MB 22kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x61f76000 @  0x7ff49ddf82a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 22.6MB 1.8MB/s 
    100% |████████████████████████████████| 51kB 21.6MB/s 
    100% |████████████████████████████████| 12.8MB 

### Sample Run

In [0]:
# Sample tagging using nltk
text = word_tokenize("Widows and orphans occur when the first line of a paragraph is the last line in a column or page")
nltk.pos_tag(text)

[('Widows', 'NNS'),
 ('and', 'CC'),
 ('orphans', 'NNS'),
 ('occur', 'VBP'),
 ('when', 'WRB'),
 ('the', 'DT'),
 ('first', 'JJ'),
 ('line', 'NN'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('paragraph', 'NN'),
 ('is', 'VBZ'),
 ('the', 'DT'),
 ('last', 'JJ'),
 ('line', 'NN'),
 ('in', 'IN'),
 ('a', 'DT'),
 ('column', 'NN'),
 ('or', 'CC'),
 ('page', 'NN')]

In [0]:
# Sample tagging using Flair
pos = SequenceTagger.load('pos-fast')
sentence = Sentence('Widows and orphans occur when the first line of a paragraph is the last line in a column or page')

# predict pos tags
pos.predict(sentence)
print(sentence.to_tagged_string())

  

2019-01-22 11:32:27,952 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.2/POS-ontonotes--h256-l1-b32-%2Bnews-forward-fast%2Bnews-backward-fast--v0.2/en-pos-ontonotes-fast-v0.2.pt not found in cache, downloading to /tmp/tmp98_gczd0


100%|██████████| 75210112/75210112 [00:03<00:00, 19786886.08B/s]

2019-01-22 11:32:32,257 copying /tmp/tmp98_gczd0 to cache at /root/.flair/models/en-pos-ontonotes-fast-v0.2.pt


2019-01-22 11:32:32,359 removing temp file /tmp/tmp98_gczd0
Widows <NNS> and <CC> orphans <NNS> occur <VBP> when <WRB> the <DT> first <JJ> line <NN> of <IN> a <DT> paragraph <NN> is <VBZ> the <DT> last <JJ> line <NN> in <IN> a <DT> column <NN> or <CC> page <NN>


### Importing Dataset

In [0]:
# file was uploaded manually to local environment
data = open('pos-tagged_corpus.txt','r')
txt = data.read()
print(txt)

-DOCSTART- -X- -X- O

SOCCER NN I-NP O
- : O O
JAPAN NNP I-NP I-LOC
GET VB I-VP O
LUCKY NNP I-NP O
WIN NNP I-NP O
, , O O
CHINA NNP I-NP I-PER
IN IN I-PP O
SURPRISE DT I-NP O
DEFEAT NN I-NP O
. . O O

Nadim NNP I-NP I-PER
Ladki NNP I-NP I-PER

AL-AIN NNP I-NP I-LOC
, , O O
United NNP I-NP I-LOC
Arab NNP I-NP I-LOC
Emirates NNPS I-NP I-LOC
1996-12-06 CD I-NP O

Japan NNP I-NP I-LOC
began VBD I-VP O
the DT I-NP O
defence NN I-NP O
of IN I-PP O
their PRP$ I-NP O
Asian JJ I-NP I-MISC
Cup NNP I-NP I-MISC
title NN I-NP O
with IN I-PP O
a DT I-NP O
lucky JJ I-NP O
2-1 CD I-NP O
win VBP I-VP O
against IN I-PP O
Syria NNP I-NP I-LOC
in IN I-PP O
a DT I-NP O
Group NNP I-NP O
C NNP I-NP O
championship NN I-NP O
match NN I-NP O
on IN I-PP O
Friday NNP I-NP O
. . O O

But CC O O
China NNP I-NP I-LOC
saw VBD I-VP O
their PRP$ I-NP O
luck NN I-NP O
desert VB I-VP O
them PRP I-NP O
in IN I-PP O
the DT I-NP O
second NN I-NP O
match NN I-NP O
of IN I-PP O
the DT I-NP O
group NN I-NP O
, , O O
crashing V

### format
data file contain one word per line with empty
lines representing sentence boundaries. At the end
of each line there is a tag which states whether the
current word is inside a named entity or not. The
tag also encodes the type of named entity. 
## Each line contains four fields: the word, its partof-speech tag, its chunk tag and its named entity tag

## Extracting Sentences

In [0]:
##text in form of list
txt = txt.split('\n')

### removing DOCSTART
txt = [x for x in txt if x != '-DOCSTART- -X- -X- O']
for i in range(10):
  print(txt[i])


SOCCER NN I-NP O
- : O O
JAPAN NNP I-NP I-LOC
GET VB I-VP O
LUCKY NNP I-NP O
WIN NNP I-NP O
, , O O
CHINA NNP I-NP I-PER
IN IN I-PP O


In [0]:
### Extracting Sentences ###

# Initialize empty list for storing words
words = []
#initialize empty list for storing sentences
corpus = []
for i in tqdm(txt):
  if i =='':
    # words form a sentence
    corpus.append(' '.join(words))
    # Empty words list
    words = []
  else:
    words.append(i.split()[0])
    
# did it work?
for i in range(10):
  print(corpus[i])

print(len(corpus))

100%|██████████| 50120/50120 [00:00<00:00, 898792.24it/s]


SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT .
Nadim Ladki
AL-AIN , United Arab Emirates 1996-12-06
Japan began the defence of their Asian Cup title with a lucky 2-1 win against Syria in a Group C championship match on Friday .
But China saw their luck desert them in the second match of the group , crashing to a surprise 2-0 defeat to newcomers Uzbekistan .
China controlled most of the match and saw several chances missed until the 78th minute when Uzbek striker Igor Shkvyrin took advantage of a misdirected defensive header to lob the ball over the advancing Chinese keeper and into an empty net .
Oleg Shatskiku made sure of the win in injury time , hitting an unstoppable left foot shot from just outside the area .
The former Soviet republic was playing in an Asian Cup finals tie for the first time .
Despite winning the Asian Games title two years ago , Uzbekistan are in the finals as outsiders .
3685


## Extracting pos

In [0]:
### Extracting Sentences ###

# Initialize empty list for storing word pos
w_pos = []
#initialize empty list for storing sentence pos
POS = []
for i in tqdm(txt):
  if i =='':
    # words form a sentence
    POS.append(' '.join(w_pos))
    # Empty words list
    w_pos = []
  else:
    w_pos.append(i.split()[1])
    
# did it work?
for i in range(10):
  print(corpus[i])
  print(POS[i])
  
print(len(POS))

100%|██████████| 50120/50120 [00:00<00:00, 816937.79it/s]



SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT .
NN : NNP VB NNP NNP , NNP IN DT NN .
Nadim Ladki
NNP NNP
AL-AIN , United Arab Emirates 1996-12-06
NNP , NNP NNP NNPS CD
Japan began the defence of their Asian Cup title with a lucky 2-1 win against Syria in a Group C championship match on Friday .
NNP VBD DT NN IN PRP$ JJ NNP NN IN DT JJ CD VBP IN NNP IN DT NNP NNP NN NN IN NNP .
But China saw their luck desert them in the second match of the group , crashing to a surprise 2-0 defeat to newcomers Uzbekistan .
CC NNP VBD PRP$ NN VB PRP IN DT NN NN IN DT NN , VBG TO DT NN CD NN TO NNS NNP .
China controlled most of the match and saw several chances missed until the 78th minute when Uzbek striker Igor Shkvyrin took advantage of a misdirected defensive header to lob the ball over the advancing Chinese keeper and into an empty net .
NNP VBD JJS IN DT NN CC VBD JJ NNS VBD IN DT JJ NN WRB NNP NN JJ NNP VBD NN IN DT JJ JJ NN TO VB DT NN IN DT VBG JJ NN CC IN DT JJ NN .
Oleg Shatskiku 

In [0]:
### Removing blanks form sentence and pos

corpus = [x for x in corpus if x!= '']
POS = [x for x in POS if x!= '']
print(type(POS[1]),len(corpus))

<class 'str'> 3453


#### corpus and pos have been extracted

## POS tagging using nltk

In [0]:
#for storing results
nltk_pos = []
#for every sentence
for i in tqdm(corpus):
  # Tokenize sentence
  text = word_tokenize(i)
  #tag Words
  z = nltk.pos_tag(text)
  # store
  nltk_pos.append(z)
  
for i in range(10):
  print(nltk_pos[i])
  
print(len(nltk_pos))

100%|██████████| 3453/3453 [00:03<00:00, 983.19it/s] 

[('SOCCER', 'NNP'), ('-', ':'), ('JAPAN', 'NNP'), ('GET', 'NNP'), ('LUCKY', 'NNP'), ('WIN', 'NNP'), (',', ','), ('CHINA', 'NNP'), ('IN', 'NNP'), ('SURPRISE', 'NNP'), ('DEFEAT', 'NNP'), ('.', '.')]
[('Nadim', 'NNP'), ('Ladki', 'NNP')]
[('AL-AIN', 'NNP'), (',', ','), ('United', 'NNP'), ('Arab', 'NNP'), ('Emirates', 'VBZ'), ('1996-12-06', 'JJ')]
[('Japan', 'NNP'), ('began', 'VBD'), ('the', 'DT'), ('defence', 'NN'), ('of', 'IN'), ('their', 'PRP$'), ('Asian', 'JJ'), ('Cup', 'NNP'), ('title', 'NN'), ('with', 'IN'), ('a', 'DT'), ('lucky', 'JJ'), ('2-1', 'JJ'), ('win', 'NN'), ('against', 'IN'), ('Syria', 'NNP'), ('in', 'IN'), ('a', 'DT'), ('Group', 'NNP'), ('C', 'NNP'), ('championship', 'NN'), ('match', 'NN'), ('on', 'IN'), ('Friday', 'NNP'), ('.', '.')]
[('But', 'CC'), ('China', 'NNP'), ('saw', 'VBD'), ('their', 'PRP$'), ('luck', 'NN'), ('desert', 'VB'), ('them', 'PRP'), ('in', 'IN'), ('the', 'DT'), ('second', 'JJ'), ('match', 'NN'), ('of', 'IN'), ('the', 'DT'), ('group', 'NN'), (',', ','), (

In [0]:
### Extracting final pos by nltk in a list ###

tmp = []
nltk_result = []

for i in tqdm(nltk_pos):
  tmp = []
  for j in i:
    tmp.append(j[1])
  nltk_result.append(' '.join(tmp))
    
for i in range(10):
  print(nltk_result[i])
  print(corpus[i])
  
print(type(nltk_result[1]))

100%|██████████| 3453/3453 [00:00<00:00, 155676.88it/s]

NNP : NNP NNP NNP NNP , NNP NNP NNP NNP .
SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT .
NNP NNP
Nadim Ladki
NNP , NNP NNP VBZ JJ
AL-AIN , United Arab Emirates 1996-12-06
NNP VBD DT NN IN PRP$ JJ NNP NN IN DT JJ JJ NN IN NNP IN DT NNP NNP NN NN IN NNP .
Japan began the defence of their Asian Cup title with a lucky 2-1 win against Syria in a Group C championship match on Friday .
CC NNP VBD PRP$ NN VB PRP IN DT JJ NN IN DT NN , VBG TO DT NN JJ NN TO NNS NNP .
But China saw their luck desert them in the second match of the group , crashing to a surprise 2-0 defeat to newcomers Uzbekistan .
NNP VBD JJS IN DT NN CC VBD JJ NNS VBN IN DT CD NN WRB NNP NN NNP NNP VBD NN IN DT JJ JJ NN TO VB DT NN IN DT VBG JJ NN CC IN DT JJ NN .
China controlled most of the match and saw several chances missed until the 78th minute when Uzbek striker Igor Shkvyrin took advantage of a misdirected defensive header to lob the ball over the advancing Chinese keeper and into an empty net .
NNP NNP VBD N

## POS tagging using flair

In [0]:
# initiating object
pos = SequenceTagger.load('pos-fast')

#for storing pos tagged string
f_pos = []

for i in tqdm(corpus):
  sentence = Sentence(i)
  pos.predict(sentence)
  f_pos.append(sentence.to_tagged_string())

100%|██████████| 3453/3453 [13:10<00:00,  2.14it/s]


In [0]:
for i in range(10):
  print(f_pos[i])
  print(corpus[i])
  
print(len(f_pos),len(corpus),len(nltk_result))

SOCCER <NNP> - <HYPH> JAPAN <NNP> GET <VBP> LUCKY <JJ> WIN <NN> , <,> CHINA <NNP> IN <IN> SURPRISE <NNP> DEFEAT <NNP> . <.>
SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT .
Nadim <NNP> Ladki <NNP>
Nadim Ladki
AL-AIN <NNP> , <,> United <NNP> Arab <NNP> Emirates <NNP> 1996-12-06 <CD>
AL-AIN , United Arab Emirates 1996-12-06
Japan <NNP> began <VBD> the <DT> defence <NN> of <IN> their <PRP$> Asian <NNP> Cup <NNP> title <NN> with <IN> a <DT> lucky <JJ> 2-1 <CD> win <NN> against <IN> Syria <NNP> in <IN> a <DT> Group <NNP> C <NNP> championship <NN> match <NN> on <IN> Friday <NNP> . <.>
Japan began the defence of their Asian Cup title with a lucky 2-1 win against Syria in a Group C championship match on Friday .
But <CC> China <NNP> saw <VBD> their <PRP$> luck <NN> desert <VB> them <PRP> in <IN> the <DT> second <JJ> match <NN> of <IN> the <DT> group <NN> , <,> crashing <VBG> to <IN> a <DT> surprise <JJ> 2-0 <CD> defeat <NN> to <IN> newcomers <NNS> Uzbekistan <NNP> . <.>
But China saw 

In [0]:
### Extracting POS tags

for i in tqdm(range(len(f_pos))):
  # for words in corpus sentence
  for j in corpus[i].split():
    # remove those words
    f_pos[i] = str(f_pos[i]).replace(j,"",1)
  # Removing < > symbols
  for j in  ['<','>']:
    f_pos[i] = str(f_pos[i]).replace(j,"")
    f_pos[i] = re.sub(' +', ' ', str(f_pos[i]))
    f_pos[i] = str(f_pos[i]).lstrip()
    
    
for i in range(10):
  print(f_pos[i])
  print(corpus[i])
    
print(type(f_pos[1]),len(corpus))

100%|██████████| 3453/3453 [00:00<00:00, 41134.53it/s]

NNP HYPH NNP VBP JJ NN , NNP IN NNP NNP .
SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT .
NNP NNP
Nadim Ladki
NNP , NNP NNP NNP CD
AL-AIN , United Arab Emirates 1996-12-06
NNP VBD DT NN IN PRP$ NNP NNP NN IN DT JJ D NN IN NNP IN DT NNP C NNP NN NN IN NNP .
Japan began the defence of their Asian Cup title with a lucky 2-1 win against Syria in a Group C championship match on Friday .
CC NNP VBD PRP$ NN VB PRP IN DT JJ NN IN DT NN , VBG IN DT JJ CD NN IN NNS NNP .
But China saw their luck desert them in the second match of the group , crashing to a surprise 2-0 defeat to newcomers Uzbekistan .
NNP VBD JJS IN DT NN CC VBD JJ NNS VBN IN DT JJ NN WRB NNP NN NNP NNP VBD NN IN DT VBN JJ NN TO VB DT NN IN DT VBG JJ NN CC IN DT JJ NN .
China controlled most of the match and saw several chances missed until the 78th minute when Uzbek striker Igor Shkvyrin took advantage of a misdirected defensive header to lob the ball over the advancing Chinese keeper and into an empty net .
NNP NNP VB

#### POS from nltk and Flair are ready

In [0]:
### Removing Symbols and redundant space
for i in tqdm(range(len(corpus))):
  #Removing Symbols
  corpus[i] = re.sub('[^a-zA-Z]', ' ', str(corpus[i]))
  POS[i] = re.sub('[^a-zA-Z]', ' ', str(POS[i]))
  f_pos[i] = re.sub('[^a-zA-Z]', ' ', str(f_pos[i]))
  nltk_result[i] = re.sub('[^a-zA-Z]', ' ', str(nltk_result[i]))
  #Removing HYPH NN
  f_pos[i] = str(f_pos[i]).replace('HYPH',"")
  f_pos[i] = str(f_pos[i]).replace('SYM',"")
  POS[i] = str(POS[i]).replace('SYM',"")
  POS[i] = str(POS[i]).replace('HYPH',"")
  nltk_result[i] = str(nltk_result[i].replace('HYPH',''))
  nltk_result[i] = str(nltk_result[i].replace('SYM',''))                      
  # Removing redundant space
  POS[i] = re.sub(' +', ' ', str(POS[i]))
  f_pos[i] = re.sub(' +', ' ', str(f_pos[i]))
  corpus[i] = re.sub(' +', ' ', str(corpus[i]))
  nltk_result[i] = re.sub(' +', ' ', str(nltk_result[i]))
  

100%|██████████| 3453/3453 [00:00<00:00, 24439.02it/s]


In [0]:
for i in range(1000):
  print('corpus   '+corpus[i])
  print('actual   '+POS[i])
  print('nltk     '+nltk_result[i])
  print('flair    '+f_pos[i])
  print('-'*50)

corpus   SOCCER JAPAN GET LUCKY WIN CHINA IN SURPRISE DEFEAT 
actual   NN NNP VB NNP NNP NNP IN DT NN 
nltk     NNP NNP NNP NNP NNP NNP NNP NNP NNP 
flair    NNP NNP VBP JJ NN NNP IN NNP NNP 
--------------------------------------------------
corpus   Nadim Ladki
actual   NNP NNP
nltk     NNP NNP
flair    NNP NNP
--------------------------------------------------
corpus   AL AIN United Arab Emirates 
actual   NNP NNP NNP NNPS CD
nltk     NNP NNP NNP VBZ JJ
flair    NNP NNP NNP NNP CD
--------------------------------------------------
corpus   Japan began the defence of their Asian Cup title with a lucky win against Syria in a Group C championship match on Friday 
actual   NNP VBD DT NN IN PRP JJ NNP NN IN DT JJ CD VBP IN NNP IN DT NNP NNP NN NN IN NNP 
nltk     NNP VBD DT NN IN PRP JJ NNP NN IN DT JJ JJ NN IN NNP IN DT NNP NNP NN NN IN NNP 
flair    NNP VBD DT NN IN PRP NNP NNP NN IN DT JJ D NN IN NNP IN DT NNP C NNP NN NN IN NNP 
--------------------------------------------------
corp

### Custom Evaluation

In [0]:
len(POS), len(f_pos), len(nltk_result)

(3453, 3453, 3453)

Word-wise

In [0]:
### EVALUATION FUNCTION
def eval2(x,y):
  count = 0
  length = 0
  # for every sentence in dataset
  for i in range(len(x)):
    # if the sentence length match
    if len(x[i].split()) == len(y[i].split()):
      # compare each word
      for j in range(len(x[i].split())):
        if x[i][j] == y[i][j] :
          count = count+1
          length = length + 1
        else:
          length = length + 1
    else:
      length = length - len(x[i].split())  
  return (count/length)*100

In [0]:
print("nltk Score    ", eval2(POS,nltk_result))
print("Flair Score   ", eval2(POS,f_pos))

nltk Score     85.38654023442645
Flair Score    90.55306768590091
